## Get the data
csv headings: id, created_at, source, original_text, clean_text, favorite_count, retweet_count, hashtags, trend <br>
hashtags format: strings with comma separated hashtags

In [1]:
#File paths
US_tweets_file = './Data/USTweets.csv'
UK_tweets_file = './Data/UKTweets.csv'

In [2]:
import csv

tweets = []
hashtags = [] #list of lists of hashtags e.g. hashtags[0] = ["hashtag1", "hashtag2"]
hashtags_strings = [] #list of hashtags string e.g. hashtags[0] = ["hashtag1, hashtag2"]


UK data

In [3]:
with open(UK_tweets_file) as data_file:
    data = csv.reader(data_file)
    next(data) #To skip the headings
    for row in data:
        tweets.append(row[4])
        hashtags.append(row[7].split(", "))
        hashtags_strings.append(row[7])

US data

In [4]:
# with open(US_tweets_file) as data_file:
#     data = csv.reader(data_file)
#     next(data) #To skip the headings
#     for row in data:
#         tweets.append(row[4])
#         hashtags.append(row[7].split(", "))
#         hashtags_strings.append(row[7])

## Initialize the tokenizers
Will use a specialized tokenizer for the hashtags because we need to encode all the hashtags. It also does not matter if the encoding of the tweets match the encoding of the hashtags.

In [5]:
from keras.preprocessing.text import Tokenizer

tweets_tokenizer = Tokenizer(oov_token="<OOV>")
tweets_tokenizer.fit_on_texts(tweets)
tweets_word_index = tweets_tokenizer.word_index
tweets_index_word = tweets_tokenizer.index_word

In [6]:
hashtags_tokenizer = Tokenizer(oov_token="<OOV>")
hashtags_tokenizer.fit_on_texts(hashtags_strings)
hashtags_word_index = hashtags_tokenizer.word_index
hashtags_index_word = hashtags_tokenizer.index_word

In [7]:
print(f'There are {len(tweets)} tweets, ')
print(f'the tweets contain {len(tweets_tokenizer.word_index)} different words.')
print(f'There are {len(hashtags_tokenizer.word_index)} different hashtags')
print('Here are the tokenized hashtags')
print(hashtags_word_index)

There are 6947 tweets, 
the tweets contain 10367 different words.
There are 2179 different hashtags
Here are the tokenized hashtags
{'<OOV>': 1, 'emergencyalert': 2, 'newtot': 3, 'forgotten80s': 4, 'malpractice': 5, 'nufc': 6, 'showman': 7, 'bouwhu': 8, 'barçaatleti': 9, 'northernlights': 10, 'aurora': 11, 'spurs': 12, 'thfc': 13, 'stgeorgesday': 14, 'derrenbrown': 15, 'davisgarcia': 16, 'auroraborealis': 17, 'coys': 18, 'mufc': 19, 'emergencyalerts': 20, 'swfc': 21, 'earthday': 22, 'dcfc': 23, 'tottenham': 24, 'premierleague': 25, 'donbelle': 26, 'barcaatleti': 27, 'facup': 28, 'pinkchella': 29, 'newcastle': 30, 'rcbvrr': 31, 'levyout': 32, 'warriors': 33, 'dubnation': 34, 'rcbvsrr': 35, 'motd2': 36, 'cskvskkr': 37, 'sundayvibes': 38, 'newcastleunited': 39, 'solarpromotionsociety': 40, 'sps': 41, 'iub': 42, 'team': 43, 'solar': 44, 'promotion': 45, 'ttid': 46, 'getwellsoonpakistan': 47, 'timelessconcert': 48, 'england': 49, 'stgeorgesday2023': 50, 'dcfcfans': 51, 'derrenbrownshowman':

## Prepare the pre-trained embeddings

In [9]:
import pickle
import numpy as np

num_tokens = len(tweets_word_index) + 1
embedding_dim = 300
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
with open('../Hashtag-recomender-embeddings/embeddings_index_object.pkl', 'rb') as embeddings_file:
    embeddings_index = pickle.load(embeddings_file)
    for word, i in tweets_word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            # This includes the representation for "padding" and "OOV"
            if embedding_vector.shape == (300,):
                embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1

In [10]:
print(f'hits:{hits}, misses: {misses}')

hits:8633, misses: 1734


## Create the sequences and pad them and one multi-hot encode the hashtags
Will use a binary vector to encode the hashtags to the model can categorize the tweets. e.g. hashtags[0] = [tag1, tag2], and tag1 has encoding of 1 and tag2 has encoding 2, then the binary vector wil be [0 1 1 0 0 ... no_of_different_hashtags]

In [11]:
sequence_length = 10

from keras.utils import pad_sequences
tweets_sequences = tweets_tokenizer.texts_to_sequences(tweets)
hashtags_sequences = hashtags_tokenizer.texts_to_sequences(hashtags)
tweets_sequences_padded = pad_sequences(tweets_sequences, padding="post", maxlen=sequence_length)

In [12]:
from keras.utils import to_categorical
import tensorflow as tf

no_of_different_hashtags = len(hashtags_word_index) + 1
no_of_hashtags = len(hashtags_sequences)

encoded_hashtags = np.zeros((no_of_hashtags, no_of_different_hashtags))

for i, hashtags_indices in enumerate(hashtags_sequences):
    encoded_hashtags[i][hashtags_indices] = 1


encoded_hashtags = np.array(encoded_hashtags)


## Split the data

In [27]:
training_split = 0.8
training_tweets_count = int(0.8 * len(tweets_sequences_padded))

5557


In [28]:
train_data = tweets_sequences_padded[0:training_tweets_count]
train_labels = encoded_hashtags[0:training_tweets_count]
test_data = tweets_sequences_padded[training_tweets_count:]
test_labels = encoded_hashtags[training_tweets_count:]

print(f'we have {len(train_data)} tweets for training and {len(test_data)} for testing')

we have 5557 tweets for training and 1390 for testing


## Build the model

In [44]:
#hyperparameters
embedding_dimensions = 300
lstm_units = 128
dropout_value = 0.5
conv_filters = 64
conv_kernel_size = 5

In [45]:
from keras import initializers

no_of_tweets_words = len(tweets_word_index) + 1

hashtag_recommender_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(no_of_tweets_words, embedding_dimensions,
                               input_length=sequence_length,
                                 embeddings_initializer=initializers.Constant(embedding_matrix),
                                   trainable=False),
    # tf.keras.layers.Conv1D(conv_filters, conv_kernel_size, activation='relu'),
    # tf.keras.layers.Dropout(dropout_value),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True)),
    # tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units)),
    tf.keras.layers.Dropout(dropout_value),
    tf.keras.layers.Dense(no_of_different_hashtags, activation='softmax')
])

hashtag_recommender_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics = ['accuracy']
)

hashtag_recommender_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 10, 300)           3110400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 256)              439296    
 nal)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 2180)              560260    
                                                                 
Total params: 4,109,956
Trainable params: 999,556
Non-trainable params: 3,110,400
_________________________________________________________________


## Train the model

In [46]:
epochs = 30
hashtag_recommender_model.fit(test_data, test_labels, epochs=epochs, validation_data=(test_data, test_labels))

Epoch 1/30
44/44 [==============================] - 4s 32ms/step - loss: 10.3009 - accuracy: 0.2583 - val_loss: 7.8024 - val_accuracy: 0.3209
Epoch 2/30
44/44 [==============================] - 1s 19ms/step - loss: 7.6739 - accuracy: 0.3576 - val_loss: 7.0572 - val_accuracy: 0.4403
Epoch 3/30
44/44 [==============================] - 1s 19ms/step - loss: 7.1406 - accuracy: 0.4719 - val_loss: 6.4289 - val_accuracy: 0.5432
Epoch 4/30
44/44 [==============================] - 1s 19ms/step - loss: 6.8951 - accuracy: 0.5173 - val_loss: 6.4537 - val_accuracy: 0.5820
Epoch 5/30
44/44 [==============================] - 1s 19ms/step - loss: 6.8495 - accuracy: 0.5597 - val_loss: 6.3712 - val_accuracy: 0.6079
Epoch 6/30
44/44 [==============================] - 1s 20ms/step - loss: 6.8886 - accuracy: 0.5957 - val_loss: 6.4351 - val_accuracy: 0.6626
Epoch 7/30
44/44 [==============================] - 1s 18ms/step - loss: 7.0276 - accuracy: 0.6245 - val_loss: 6.1575 - val_accuracy: 0.6719
Epoch 8/30
4

## Get hashtags!!

In [ ]:
def predict(tweet, tweet_tokenizer, hashtag_tokenizer, pad_length, model):
    tweet_sequence = tweet_tokenizer.texts_to_sequences([tweet])[0]
    padded_tweet_sequence = pad_sequences([tweet_sequence], maxlen=pad_length, padding='post')
    prediction = (model.predict(padded_tweet_sequence))
    hashtag_indices = np.argsort(prediction, axis=-1)[0][-3:]
    return [hashtag_tokenizer.index_word[hashtag_index] for hashtag_index in hashtag_indices]


In [ ]:
print(predict('lit derby gg', tweets_tokenizer, hashtags_tokenizer, 10, hashtag_recommender_model))